In [1]:
import re
import zipfile
import numpy as np
import pathlib
import rasterio
import geopandas as gpd
import pandas as pd

from food_security import data_reader
from rasterio.transform import from_origin, Affine, from_bounds
from rasterio.mask import mask
from rasterio.io import MemoryFile

import matplotlib.pyplot as plt

In [2]:
src_dir = pathlib.Path("~").expanduser().resolve() / "data/food-security"
github_dir = (
    pathlib.Path("~").expanduser().resolve() / "projects/food-security/Food-Security/"
)

In [3]:
communes_gdf = gpd.read_file(src_dir / "provinces_area.shp")
communes_gdf

,Code,Name,area,perimeter,area [ha],geometry
0,AD01,An Giang,3.684739e+09,322721.990577,368474,"POLYGON ((105.11524 10.95566, 105.11463 10.946..."
1,AD01,Bac Lieu,2.559375e+09,302354.696741,255937,"POLYGON ((105.32591 9.60004, 105.32755 9.59998..."
2,AD01,Ben Tre,2.434102e+09,245574.864110,243410,"POLYGON ((106.42508 10.32019, 106.44474 10.308..."
3,AD01,Ca Mau,5.335406e+09,416302.040508,533541,"MULTIPOLYGON (((104.87929 8.38103, 104.87884 8..."
4,AD01,Can Tho,1.494663e+09,207548.702404,149466,"POLYGON ((105.59811 10.23951, 105.60244 10.234..."
5,AD01,Dong Thap,3.526706e+09,360735.590132,352671,"POLYGON ((105.44963 10.95591, 105.45154 10.955..."
6,AD01,Hau Giang,1.682172e+09,227180.904870,168217,"POLYGON ((105.5791 9.98585, 105.58078 9.98491,..."
7,AD01,Kien Giang,6.539146e+09,954257.153251,653915,"MULTIPOLYGON (((103.4391 9.26394, 103.43816 9...."
8,AD01,Long An,4.698667e+09,485008.827402,469867,"POLYGON ((105.81728 10.96027, 105.85563 10.911..."
9,AD01,Soc Trang,3.367090e+09,298543.976511,336709,"POLYGON ((105.90627 9.93254, 105.90919 9.93225..."


In [4]:
def create_population_raster(zip_file, tif_file, tfw_file):
    # Get correct transform from tfw file. Order seems to be messed up.
    with zipfile.ZipFile(zip_file, "r") as z:
        with z.open(tfw_file) as f:
            lines = f.read().decode().splitlines()
            a = float(lines[0])
            b = float(lines[1])
            c = float(lines[4])
            d = float(lines[2])
            e = float(lines[3])
            f_ = float(lines[5])

    transform = Affine(a, b, c, d, e, f_)

    with rasterio.open(f"zip+file://{zip_file}!/{tif_file}") as src:
        data = src.read()

        # Update transform of tif, since there seems to be an error in the original transform
        profile = src.profile
        profile.update({"transform": transform})

    # Create and write to a tif file in memory
    raster_file = MemoryFile()
    dst = raster_file.open(**profile)
    dst.write(data[0, ...], 1)

    return dst


def read_zipped_tif_file(zip_file, tif_file):
    # print(f'zip+file://{zip_file}!/{tif_file}')
    src = rasterio.open(f"zip+file://{zip_file}!/{tif_file}")
    return src

In [5]:
gdp_files = {
    "SSP1": {
        "zip": "/Users/hemert/data/food-security/ssp/7898409/SSP1.zip",
        "tif": "SSP1/GDP2050_ssp1.tif",
    },
    "SSP2": {
        "zip": "/Users/hemert/data/food-security/ssp/7898409/SSP2.zip",
        "tif": "SSP2/GDP2050_ssp2.tif",
    },
    "SSP3": {
        "zip": "/Users/hemert/data/food-security/ssp/7898409/SSP3.zip",
        "tif": "SSP3/GDP2050_ssp3.tif",
    },
    "SSP4": {
        "zip": "/Users/hemert/data/food-security/ssp/7898409/SSP4.zip",
        "tif": "SSP4/GDP2050_ssp4.tif",
    },
    "SSP5": {
        "zip": "/Users/hemert/data/food-security/ssp/7898409/SSP5.zip",
        "tif": "SSP5/GDP2050_ssp5.tif",
    },
}
pop_files = {
    "SSP1": {
        "zip": "/Users/hemert/data/food-security/ssp/19608594/SPP1.zip",
        "tif": "SPP1/SSP1_2050.tif",
        "tfw": "SPP1/SSP1_2050.tfw",
    },
    "SSP2": {
        "zip": "/Users/hemert/data/food-security/ssp/19608594/SSP2.zip",
        "tif": "SPP2/SSP2_2050.tif",
        "tfw": "SPP2/SSP2_2050.tfw",
    },
    "SSP3": {
        "zip": "/Users/hemert/data/food-security/ssp/19608594/SSP3.zip",
        "tif": "SPP3/SSP3_2050.tif",
        "tfw": "SPP3/SSP3_2050.tfw",
    },
    "SSP4": {
        "zip": "/Users/hemert/data/food-security/ssp/19608594/SSP4.zip",
        "tif": "SPP4/SSP4_2050.tif",
        "tfw": "SPP4/SSP4_2050.tfw",
    },
    "SSP5": {
        "zip": "/Users/hemert/data/food-security/ssp/19608594/SSP5.zip",
        "tif": "SPP5/SSP5_2050.tif",
        "tfw": "SPP5/SSP5_2050.tfw",
    },
}

In [6]:
def zonal_statistics(geometry, raster):
    masked_raster = mask(raster, geometry, nodata=np.nan, all_touched=False)[0]
    geometry_sum = np.nansum(masked_raster)

    return geometry_sum

In [7]:
SSPs = ['SSP1', 'SSP2', 'SSP3', 'SSP4', 'SSP5']
names = communes_gdf["Name"].values
df_dict = {
    'ssp': [],
    'area_name': [],
    'GDP': [],
    'population': [],
}
for ssp in SSPs:
    print(ssp)
    gdp_raster = read_zipped_tif_file(gdp_files[ssp]['zip'], gdp_files[ssp]['tif'])
    pop_raster = create_population_raster(pop_files[ssp]['zip'], pop_files[ssp]['tif'], pop_files[ssp]['tfw'])

    for name in names:
        commune_geometry = communes_gdf[communes_gdf["Name"] == name]["geometry"]

        gdp = zonal_statistics(commune_geometry, gdp_raster)
        pop = zonal_statistics(commune_geometry, pop_raster)

        df_dict['ssp'].append(ssp)
        df_dict['area_name'].append(name)
        df_dict['GDP'].append(gdp)
        df_dict['population'].append(pop)

df = pd.DataFrame(df_dict)

SSP1
SSP2
SSP3
SSP4
SSP5


In [8]:
df.to_csv(src_dir / 'ssp_gdp_and_pop.csv')

In [19]:
df

,ssp,area_name,GDP,population
0,SSP1,An Giang,4.704069e+09,2.393372e+06
1,SSP1,Bac Lieu,2.469689e+09,1.010742e+06
2,SSP1,Ben Tre,2.598716e+09,1.266103e+06
3,SSP1,Ca Mau,3.494960e+09,1.259917e+06
4,SSP1,Can Tho,9.378998e+09,1.529421e+06
...,...,...,...,...
60,SSP5,Long An,3.150922e+10,1.643388e+06
61,SSP5,Soc Trang,2.600260e+09,1.502076e+06
62,SSP5,Tien Giang,2.840182e+10,1.838524e+06
63,SSP5,Tra Vinh,2.235745e+09,1.123235e+06
